<a href="https://colab.research.google.com/github/Chinmay-47/LRDE_Deep_Learning_Project/blob/master/GAN_FINAL_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, LayerNormalization
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
import os
import matplotlib.pyplot as plt
import cv2
import sys
from PIL import Image
import numpy as np
import tensorflow as tf

# Comprhensive mode failure at around 2000 epochs

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 32
        self.weight_decay = 1e-4

        optimizer = Adam(0.0001 , 0.5)
        optimizer1 = Adam(0.0001 , 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer1)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(4*4*128 , activation = 'linear' , input_dim = self.latent_dim))
        model.add(Reshape((4,4,128)))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2))
        model.add(UpSampling2D()) 
        model.add(Conv2DTranspose(filters = 64 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2))
        model.add(UpSampling2D())
        model.add(Conv2DTranspose(filters = 32 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2))
        model.add(UpSampling2D()) 
        model.add(Conv2DTranspose(filters = 16 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2))
        model.add(UpSampling2D()) 
        model.add(Conv2DTranspose(filters = 8 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Conv2D(filters = 8 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2)) 
        model.add(Conv2D(filters = 4 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2)) 
        model.add(Conv2D(filters = 1 , kernel_size = 5 , padding = 'same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha = 0.2)) 

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()
        # model.add(Conv2DTranspose(4, kernel_size=5, input_shape=self.img_shape, padding="same"))
        # model.add(BatchNormalization()) ##
        # model.add(LeakyReLU(alpha = 0.2))
        # model.add(Conv2DTranspose(8, kernel_size=5, input_shape=self.img_shape, padding="same"))
        # model.add(BatchNormalization()) ##
        # model.add(LeakyReLU(alpha = 0.2))
        # model.add(Conv2D(8, kernel_size=5, input_shape=self.img_shape, padding="same"))
        # model.add(BatchNormalization()) ##
        # model.add(LeakyReLU(alpha = 0.2))
        model.add(Conv2D(16, kernel_size=5, input_shape=self.img_shape, padding="same"))
        model.add(BatchNormalization()) #
        model.add(LeakyReLU(alpha = 0.2))
        model.add(MaxPool2D())
        model.add(Conv2D(32, kernel_size=5, padding="same"))
        model.add(BatchNormalization()) #
        model.add(LeakyReLU(alpha = 0.2))
        model.add(MaxPool2D())
        model.add(Conv2D(64, kernel_size=5, padding="same"))
        model.add(BatchNormalization()) #
        model.add(LeakyReLU(alpha = 0.2))
        model.add(MaxPool2D())
        model.add(Conv2D(128, kernel_size=5, padding="same"))
        model.add(BatchNormalization()) #
        model.add(LeakyReLU(alpha = 0.2))
        model.add(MaxPool2D())
        model.add(Flatten())
        model.add(Dense(1 , activation = 'sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)



    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        dirname = '/content/drive/My Drive/DSET_TRIAL_100/3/'
        X_train = []
        for filename in os.listdir(dirname):
          img = Image.open(dirname + filename)
          img = np.asarray(img)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          X_train.append(img)
      
        # Rescale -1 to 1
        for x in X_train:
          x = x / 127.5 - 1
 
        X_train = np.expand_dims(X_train, axis=3)
        print(np.shape(X_train))

        X_train = np.asarray(X_train)
        # Adversarial ground truths
        
        # valid = np.ones((batch_size, 1))
        valid = []
        valid = [0.9 for i in range(batch_size)]
        valid = np.expand_dims(valid , axis=1)
        print(valid.shape)
        # fake = np.zeros((batch_size, 1))
        fake = []
        fake = [0.1 for i in range(batch_size)]
        fake = np.expand_dims(fake , axis=1)
        print(fake.shape)


        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 50*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
            if epoch % 20 == 0:
                self.generator.save('/content/drive/My Drive/GEN_MODELS_1/gen_' + str(epoch) + '.h5')

    def save_imgs(self, epoch):
        noise = np.random.normal(0, 1, (5, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5


        cnt = 0 
        for img in gen_imgs:
          img = np.squeeze(img , axis=2)
          # print(np.shape(img))
          plt.imsave("/content/drive/My Drive/GAN_IMGS/" + str(epoch) + '_' + str(cnt) + ".png" , img , cmap='gray')
          cnt+=1


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=2000, batch_size=8, save_interval=10)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 64, 64, 16)        416       
_________________________________________________________________
batch_normalization_36 (Batc (None, 64, 64, 16)        64        
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 32)        12832     
_________________________________________________________________
batch_normalization_37 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 32, 32, 32)       

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
